In [ ]:
processor = WhisperProcessor.from_pretrained(model_id)

def preprocess(batch):
    batch["input_features"] = processor(batch["audio"]["array"], sampling_rate=16000, return_tensors="np").input_features[0]
    return batch

dataset_processed = librispeech.map(preprocess, remove_columns=librispeech.column_names)

In [ ]:
model = WhisperForConditionalGeneration.from_pretrained(model_id)
model.to("cuda")
model.eval()
model.half();

In [ ]:
dataloader = DataLoader(dataset_processed.with_format("torch"), batch_size=1)

In [ ]:
for batch in tqdm(dataloader):
    input_features = batch["input_features"].to("cuda").half()
    pred_ids = model.generate(input_features, max_length=128)

In [ ]:
model = FlaxWhisperForConditionalGeneration.from_pretrained(model_id, dtype=jnp.float16, from_pt=True)

In [ ]:
dataloader = DataLoader(dataset_processed.with_format("numpy"), batch_size=1)

In [ ]:
jit_generate = jax.jit(model.generate, static_argnames=["max_length"])

In [ ]:
batch = next(iter(dataloader))
input_features = jnp.array(batch["input_features"], dtype=jnp.float16)
pred_ids = jit_generate(input_features, max_length=128)

In [ ]:
for batch in tqdm(dataloader):
    input_features = jnp.array(batch["input_features"], dtype=jnp.float16)
    pred_ids = jit_generate(input_features, max_length=128)